Let's install UMAP, a dimension reduction package we want to use to build a scatter plot.

In [1]:
!pip install --quiet umap-learn
print('UMAP installed.')

UMAP installed.


Let's load up all the data and change the type of our target variable. Let's also do a tiny bit of feature engineering; we tried introducing dummies for several variables, and only introducing dummies for the thall variable improved the accuracy of our model.

Let's also introduce a standard scaler for our four floating-point variables, plus age.

In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

HEART = '/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv'

df = pd.read_csv(filepath_or_buffer=HEART)
df['output'] = df['output'] == 1
df = pd.get_dummies(data=df, columns=['thall'])
preprocessor = ColumnTransformer(
    [
        ('age', StandardScaler(), ['age']),
        ('trtbps', StandardScaler(), ['trtbps']),
        ('chol', StandardScaler(), ['chol']),
        ('thalachh', StandardScaler(), ['thalachh']),
        ('oldpeak', StandardScaler(), ['oldpeak']),
    ],
    verbose_feature_names_out=False,
)
df[['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']] = preprocessor.fit_transform(X=df.drop(columns=['output']), y=df['output'])
columns = df.drop(columns=['output']).columns
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,output,thall_0,thall_1,thall_2,thall_3
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0,1.087338,0,0,True,False,True,False,False
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,True,False,False,True,False
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0,0.310912,2,0,True,False,False,True,False
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0,-0.206705,2,0,True,False,False,True,False
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1,-0.379244,2,0,True,False,False,True,False


Let's add an exploratory graph of each of our (scaled) numerical variables broken out by the output variable.

In [3]:
from plotly import express

for x in ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']:
    express.histogram(data_frame=df, x=x, facet_col='output').show()

Are the classes balanced in our target variable?

In [4]:
df['output'].value_counts(dropna=False).to_dict()

{True: 165, False: 138}

Let's use dimension reduction to make a scatter plot. If we see a lot of clustering then our model will probably do well; if our scatter plot looks kind of random our model will probably not do well.

In [5]:
import arrow
from umap import UMAP

time_start = arrow.now()
umap = UMAP(random_state=2024, verbose=False, n_jobs=1, low_memory=False, n_epochs=201)
df[['x', 'y']] = umap.fit_transform(X=df[columns])
print('done with UMAP in {}'.format(arrow.now() - time_start))

done with UMAP in 0:00:09.473044


In [6]:
from plotly import express
express.scatter(data_frame=df, x='x', y='y', color='output', height=600)

What do we see? We see a broad separation betwween True and False values, but we see some mixing too; there are a lot of points that have nearest neighbors that are in the other class. Let's build a model.

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df[columns], df['output'], test_size=0.2, random_state=2024, stratify=df['output'])

logreg = LogisticRegression(max_iter=10000, tol=1e-12).fit(X_train, y_train)
print('model fit in {} iterations'.format(logreg.n_iter_[0]))
print('accuracy: {:5.4f}'.format(accuracy_score(y_true=y_test, y_pred=logreg.predict(X=X_test))))
print('f1: {:5.4f}'.format(f1_score(average='weighted', y_true=y_test, y_pred=logreg.predict(X=X_test), zero_division=0)))
print(classification_report(y_true=y_test, y_pred=logreg.predict(X=X_test), zero_division=0))

model fit in 34 iterations
accuracy: 0.8852
f1: 0.8851
              precision    recall  f1-score   support

       False       0.89      0.86      0.87        28
        True       0.88      0.91      0.90        33

    accuracy                           0.89        61
   macro avg       0.89      0.88      0.88        61
weighted avg       0.89      0.89      0.89        61



Before we go let's look at the regression coefficients.

In [8]:
express.histogram(x=columns, y=logreg.coef_[0])

What are the most important features? Sex, chest pain, exercise-induced angina, and thall, whatever that is. We have broken thall out into four variables, and only two of them have large regression coefficients.

Let's look at the prediction probabilities from our model.

In [9]:
probability_df = pd.DataFrame(data=logreg.predict_proba(X=X_test), columns=['pr(False)', 'pr(True)'])
probability_df['prediction'] = logreg.predict(X=X_test)
probability_df['actual'] = y_test
probability_df['actual'] = probability_df['actual'] == True
probability_df.head()

,pr(False),pr(True),prediction,actual
0,0.363638,0.636362,True,False
1,0.024220,0.975780,True,False
2,0.992179,0.007821,False,False
3,0.892710,0.107290,False,True
4,0.032337,0.967663,True,True


Obviously pr(False) + pr(True) = 1, so we expect our data to fall on a line.

In [10]:
express.scatter(data_frame=probability_df, x='pr(False)', y='pr(True)', color='prediction')

Of course our model probabilities are nicely separated. How do they look if we color them according to the actual values?

In [11]:
express.scatter(data_frame=probability_df, x='pr(False)', y='pr(True)', color='actual')

Our model is sometimes quite confident about cases where it is wrong.